In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import datetime
import time
import math
import warnings
warnings.filterwarnings("ignore")
import glob

In [3]:
def read_label():
    label = {}
    for i in range(1, 6):
        hi = 'data/ukdale/house_{}/labels.dat'.format(i)
        label[i] = {}
        with open(hi) as f:
            for line in f:
                splitted_line = line.split(' ')
                label[i][int(splitted_line[0])] = splitted_line[1].strip() + '_' + splitted_line[0]
        label[i]['mains'] = 'mains'
    return label
labels = read_label()
# for i in range(1,3):
#     print('House {}: '.format(i), labels[i], '\n')
for i in range(1,2):
    print('House {}: '.format(i), labels[i], '\n')

House 1:  {1: 'aggregate_1', 2: 'boiler_2', 3: 'solar_thermal_pump_3', 4: 'laptop_4', 5: 'washing_machine_5', 6: 'dishwasher_6', 7: 'tv_7', 8: 'kitchen_lights_8', 9: 'htpc_9', 10: 'kettle_10', 11: 'toaster_11', 12: 'fridge_12', 13: 'microwave_13', 14: 'lcd_office_14', 15: 'hifi_office_15', 16: 'breadmaker_16', 17: 'amp_livingroom_17', 18: 'adsl_router_18', 19: 'livingroom_s_lamp_19', 20: 'soldering_iron_20', 21: 'gigE_&_USBhub_21', 22: 'hoover_22', 23: 'kitchen_dt_lamp_23', 24: 'bedroom_ds_lamp_24', 25: 'lighting_circuit_25', 26: 'livingroom_s_lamp2_26', 27: 'iPad_charger_27', 28: 'subwoofer_livingroom_28', 29: 'livingroom_lamp_tv_29', 30: 'DAB_radio_livingroom_30', 31: 'kitchen_lamp2_31', 32: 'kitchen_phone&stereo_32', 33: 'utilityrm_lamp_33', 34: 'samsung_charger_34', 35: 'bedroom_d_lamp_35', 36: 'coffee_machine_36', 37: 'kitchen_radio_37', 38: 'bedroom_chargers_38', 39: 'hair_dryer_39', 40: 'straighteners_40', 41: 'iron_41', 42: 'gas_oven_42', 43: 'data_logger_pc_43', 44: 'childs_ta

In [4]:
def read_merge_data(house):
    path = 'data/ukdale/house_{}/'.format(house)
    
    file = path + 'channel_1.dat'
    df = pd.read_table(file, sep = ' ', names = ['unix_time', labels[house][1]], 
                                       dtype = {'unix_time': 'int64', labels[house][1]:'float64'}) 
    df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    
#     file = path + 'mains.dat'
#     df = pd.read_table(file, sep = ' ', names = ['unix_time', 'mains_active_p_w', 'mains_apparent_p_va', 'mains_rms_v' ], 
#                                        dtype = {'unix_time': 'float64', 'mains_active_p_w':'float64', 'mains_apparent_p_va': 'float64', 'mains_rms_v': 'float64'}) 
#     df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    
#     num_apps = len(glob.glob(path + 'channel_*[0-9].dat'))
#     for i in range(1, num_apps + 1):
    # tv_7, fridge_12, microwave_13, gas_oven_42
    for i in [7, 12, 13, 42]:
        file = path + 'channel_{}.dat'.format(i)
        data = pd.read_table(file, sep = ' ', names = ['unix_time', labels[house][i]], 
                                       dtype = {'unix_time': 'int64', labels[house][i]:'float64'})
#         data['timestamp'] = data['unix_time'].astype("datetime64[s]")
        df = pd.merge_asof(df, data, on = 'timestamp', by='timestamp', tolerance=pd.Timedelta('4s'))
#         df = pd.merge(df, data, how = 'inner', on = 'unix_time')
    df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    df = df.set_index(df['timestamp'].values)
    df.drop(['unix_time','timestamp'], axis=1, inplace=True)
    return df
df = {}
# for i in range(1,3):
#     df[i] = read_merge_data(i)
for i in range(1,2):
    df[i] = read_merge_data(i)

In [7]:
for i in range(1,2):
    print('House {} data has shape: '.format(i), df[i].shape)
    display(df[i].tail(10))

House 1 data has shape:  (0, 5)


,aggregate_1,tv_7,fridge_12,microwave_13,gas_oven_42


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback


from timeit import default_timer as timer

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def compile_model(model):
    # output the model's structure
    model.summary()

    # Model compilation
    optimizer = Adam(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    time_cb = TimingCallback()
    callbacks = [time_cb]

    # Model Training
    history = model.fit(x_train, y_onehot_train, epochs=10, batch_size=100, validation_data=(x_test, y_onehot_test), callbacks=callbacks)

    return history, sum(time_cb.logs)


model = Sequential()
model.add(Dense(1000, activation='relu', input_dim = 784))
model.add(Dense(1000, activation='relu'))
model.add(Dense(10,activation='softmax'))



history, time_spent = compile_model(model)
view_model_results(model, history)
print('Time spent', time_spent)
